In [1]:
# setting the random seed for reproducibility
import random
random.seed(493)

# for manipulating dataframes
import pandas as pd
import numpy as np

# for statistical testing
from scipy import stats

# for working with timestamps
from datetime import datetime
from dateutil.parser import parse

# for visualizations
%matplotlib inline
import matplotlib.pyplot as plt

# to print out all the outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Executive Summary

> **Something goes here.**

### PLANNING

- [X] Planning
- [X] Acquire data
- [ ] Prepare the data
- [ ] Explore the data
- [ ] Modeling

In [2]:
def show_missing(df):
    """
    Takes a dataframe and returns a dataframe with stats
    on missing and null values with their percentages.
    """
    null_count = df.isnull().sum()
    null_percentage = (null_count / df.shape[0]) * 100
    empty_count = pd.Series(((df == ' ') | (df == '')).sum())
    empty_percentage = (empty_count / df.shape[0]) * 100
    nan_count = pd.Series(((df == 'nan') | (df == 'NaN')).sum())
    nan_percentage = (nan_count / df.shape[0]) * 100
    dfx = pd.DataFrame({'num_missing': null_count, 'missing_percentage': null_percentage,
                         'num_empty': empty_count, 'empty_percentage': empty_percentage,
                         'nan_count': nan_count, 'nan_percentage': nan_percentage})
    return dfx

In [3]:
def get_values(df, columns):
    """
    Take a dataframe and a list of columns and
    returns the value counts for the columns.
    """
    for column in columns:
        print(column)
        print('=====================================')
        print(df[column].value_counts(dropna=False))
        print('\n')

def show_values(df, param):
    if param == 'all':
        get_values(df, df.columns)
    else:
        get_values(df, param)

### ACQUIRE DATA

In [4]:
# Read csv files
peims_df = pd.read_csv('../data/in/2007-2021-summaried-peims-financial-data.csv')
staar_df1 = pd.read_csv('../data/in/tidy_campstaar1_2012to2019.csv')
staar_df2 = pd.read_csv('../data/in/tidy_campstaar2_2013to2019.csv')

C:\Users\Dd\AppData\Local\Temp\ipykernel_20844\590553161.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  staar_df1 = pd.read_csv('../data/in/tidy_campstaar1_2012to2019.csv')


In [5]:
peims_df.head()
staar_df1.head()
staar_df2.head()

,DISTRICT NUMBER,DISTRICT NAME,YEAR,GEN FUNDS-LOCAL TAX REVENUE FROM M&O,ALL FUNDS-LLOCAL TAX REVENUE FROM M&O,GEN FUNDS-STATE REVENUE,ALL FUNDS-STATE REVENUE,GEN FUNDS-FEDERAL REVENUE,ALL FUNDS-FEDERAL REVENUE,GEN FUNDS-OTHER LOCAL & INTERMEDIATE REVENUE,ALL FUNDS-OTHER LOCAL & INTERMEDIATE REVENUE,GEN FUNDS-TOTAL OPERATING REVENUE,ALL FUNDS-TOTAL OPERATING REVENUE,GEN FUNDS-LOCAL PROPERTY TAXES FROM I&S,ALL FUNDS-LOCAL PROPERTY TAXES FROM I&S,GEN FUNDS-STATE DEBT FUNDS,ALL FUNDS-STATE DEBT FUNDS,GEN FUNDS-OTHER RECEIPTS,ALL FUNDS-OTHER RECEIPTS,GEN FUNDS-OTHER REVENUE,ALL FUNDS-OTHER REVENUE,GEN FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE,ALL FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE,GEN FUNDS-EQUITY TRANSFERS,ALL FUNDS-EQUITY TRANSFERS,GEN FUNDS-TOT DEBT SERV FIN AND TRS EST REV,ALL FUNDS-TOT DEBT SERV FIN AND TRS EST REV,GEN FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE AND RECPATUR,ALL FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE AND RECPATUR,GEN FUNDS-DEBT SERVICE FINANCING RELATED REVENUE,ALL FUNDS-DEBT SERVICE FINANCING RELATED REVENUE,GEN FUNDS-ESTIMATED STATE TRS CONTRINUTIONS,ALL FUNDS-ESTIMATED STATE TRS CONTRINUTIONS,GEN FUNDS-TOTAL DEBT SERVICE FINANCING AND TRS ESTIMATE REVENUE,ALL FUNDS-TOTAL DEBT SERVICE FINANCING AND TRS ESTIMATE REVENUE,"GEN FUNDS-TOTAL OPERATING, OTR, DEBT SERV FIN, AND TRS EST REVEN","ALL FUNDS-TOTAL OPERATING, OTR, DEBT SERV FIN, AND TRS EST REVEN",GEN FUNDS-TOTAL PAYROLL EXPENDITURES,ALL FUNDS-TOTAL PAYROLL EXPENDITURES,GEN FUNDS-TOTAL PROFESSIONAL & CONTRACTED SERVICES EXPENDITURES,ALL FUNDS-TOTAL PROFESSIONAL & CONTRACTED SERVICES EXPENDITURES,GEN FUNDS-TOTAL SUPPLIES & MATERIALS EXPENDITURES,ALL FUNDS-TOTAL SUPPLIES & MATERIALS EXPENDITURES,GEN FUNDS-TOTAL OTHER OPERATING EXPENDITURES,ALL FUNDS-TOTAL OTHER OPERATING EXPENDITURES,GEN FUNDS-TOTAL OPERATING EXPENDITURES BY OBJ,ALL FUNDS-TOTAL OPERATING EXPENDITURES BY OBJ,GEN FUNDS-TOTAL DEBT SERVICE EXPEND BY OBJ,ALL FUNDS-TOTAL DEBT SERVICE EXPEND BY OBJ,GEN FUNDS-TOTAL CAPITAL OUTLAY EXPEND BY OBJ,ALL FUNDS-TOTAL CAPITAL OUTLAY EXPEND BY OBJ,GEN FUNDS-TOTAL NON-OPER EXPENDITURES BY OBJ,ALL FUNDS-TOTAL NON-OPER EXPENDITURES BY OBJ,GEN FUNDS-TOTAL NON-OPER AND OPER OEXPENDITURES BY OBJ,ALL FUNDS-TOTAL NON-OPER AND OPER OEXPENDITURES BY OBJ,"GEN FUNDS-INSTRUCTION + TRANSFER EXPEND-FCT11,95","ALL FUNDS-INSTRUCTION + TRANSFER EXPEND-FCT11,95","GEN FUNDS-INSTRUC RESOURCE MEDIA SERVICE EXP, FCT12","ALL FUNDS-INSTRUC RESOURCE MEDIA SERVICE, FCT12","GEN FUNDS-CURRICULUM/STAFF DEVELOPMENT EXP, FCT13","ALL FUNDS-CURRICULUM/STAFF DEVELOPMENT EXP, FCT13","GEN FUNDS-INSTRUC LEADERSHIP EXPEND, FCT21","ALL FUNDS-INSTRUC LEADERSHIP EXPEND, FCT21","GEN FUNDS-CAMPUS ADMINISTRATION EXPEND, FCT23","ALL FUNDS-CAMPUS ADMINISTRATION EXPEND, FCT23","GEN FUNDS-GUIDANCE & COUNSELING SERVICES EXP, FCT31","ALL FUNDS-GUIDANCE 7 COUNSELING SERVICES EXP, FCT31","GEN FUNDS-SOCIAL WORK SERVICES EXP, FCT32","ALL FUNDS-SOCIAL WORK SERVICES EXP, FCT32","GEN FUNDS-HEALTH SERVICES EXP, FCT33","ALL FUNDS-HEALTH SERVICES EXP, FCT33","GEN FUNDS-TRANSPORTATION EXPENDITURES, FCT34","ALL FUNDS-TRANSPORTATION EXPENDITURES, FCT34","GEN FUNDS-FOOD SERVICE EXPENDITURES, FCT35","ALL FUNDS-FOOD SERVICE EXPENDITURES, FCT35","GEN FUNDS-EXTRACURRICULAR EXPENDITURES, FCT36","ALL FUNDS-EXTRACURRICULAR EXPENDITURES,FCT36","GEN FUNDS-GENERAL ADMINISTRAT EXPEND-FCT41,80,92","ALL FUNDS-GENERAL ADMINISTRAT EXPEND-FCT41,80,92","GEN FUNDS-PLANT MAINTENANCE/OPERA EXPEND, FCT51","ALL FUNDS-PLANT MAINTENANCE/OPERA EXPEND, FCT51","GEN FUNDS-SECURITY/MONITORING SERVICE EXPEND, FCT52","ALL FUNDS-SECURITY/MONITORING SERVICE EXPEND, FCT5","GEN FUNDS-DATA PROCESSING SERVICES EXPEND, FCT53","ALL FUNDS-DATA PROCESSING SERVICES, FCT53","GEN FUNDS-COMMUNITY SERVICES, FCT61","ALL FUNDS-COMMUNITY SERVICES, FCT61",GEN FUNDS-TOTAL OPERATE EXPEND BY FUNCTION,ALL FUNDS-TOTAL OPERATE EXPEND BY FUNCTION,GEN FUNDS-NON-OPER EXP BY FUNCTION(1X-9X)(65XX),ALL FUNDS-NON-OPER EXP BY FUNCTION(1X-9X)(65XX),GEN 

,data_release,data_category,data_level,release_year,test_year,campus_number,grade_level,subject,proficiency,demog,numerator,denominator,new_rate
0,tapr,CAMPSTAAR1.csv,campus,2014,2013,1902001,eoc,algebra_1,approaches,all_students,41,48,85.42
1,tapr,CAMPSTAAR1.csv,campus,2014,2014,1902001,eoc,algebra_1,approaches,all_students,51,57,89.47
2,tapr,CAMPSTAAR1.csv,campus,2014,2013,1902001,eoc,algebra_1,approaches,at_risk,11,17,64.71
3,tapr,CAMPSTAAR1.csv,campus,2014,2014,1902001,eoc,algebra_1,approaches,at_risk,32,38,84.21
4,tapr,CAMPSTAAR1.csv,campus,2014,2013,1902001,eoc,algebra_1,approaches,economic_disadvant,17,20,85.00


,data_release,data_category,data_level,release_year,test_year,campus_number,grade_level,subject,proficiency,demog,numerator,denominator,new_rate
0,tapr,CAMPSTAAR2.csv,campus,2019,2019,1902001,eoc,english_i,approaches,all_students,40,51,78.43
1,tapr,CAMPSTAAR2.csv,campus,2019,2019,1903001,eoc,english_i,approaches,all_students,88,127,69.29
2,tapr,CAMPSTAAR2.csv,campus,2019,2019,1904001,eoc,english_i,approaches,all_students,50,72,69.44
3,tapr,CAMPSTAAR2.csv,campus,2019,2019,1906002,eoc,english_i,approaches,all_students,21,28,75.00
4,tapr,CAMPSTAAR2.csv,campus,2019,2019,1907001,eoc,english_i,approaches,all_students,245,382,64.14


### PREPARATION

In [6]:
peims_df.shape
staar_df1.shape
staar_df2.shape

(18213, 150)

(6249982, 13)

(10024061, 13)

In [7]:
staar_df = pd.concat([staar_df1, staar_df2])

In [8]:
staar_df.head()

,data_release,data_category,data_level,release_year,test_year,campus_number,grade_level,subject,proficiency,demog,numerator,denominator,new_rate
0,tapr,CAMPSTAAR1.csv,campus,2014,2013,1902001,eoc,algebra_1,approaches,all_students,41,48,85.42
1,tapr,CAMPSTAAR1.csv,campus,2014,2014,1902001,eoc,algebra_1,approaches,all_students,51,57,89.47
2,tapr,CAMPSTAAR1.csv,campus,2014,2013,1902001,eoc,algebra_1,approaches,at_risk,11,17,64.71
3,tapr,CAMPSTAAR1.csv,campus,2014,2014,1902001,eoc,algebra_1,approaches,at_risk,32,38,84.21
4,tapr,CAMPSTAAR1.csv,campus,2014,2013,1902001,eoc,algebra_1,approaches,economic_disadvant,17,20,85.00


In [9]:
staar_df.shape

(16274043, 13)

In [10]:
staar_df = staar_df.drop_duplicates(keep='first')

In [11]:
staar_df.shape

(15116278, 13)

In [12]:
show_values(staar_df, ['data_release', 'data_category', 'data_level',
                       'release_year', 'test_year', 'grade_level', 'subject',
                       'proficiency', 'demog'
                      ])

data_release
tapr    15116278
Name: data_release, dtype: int64


data_category
CAMPSTAAR2.csv    8866296
CAMPSTAAR1.csv    6249982
Name: data_category, dtype: int64


data_level
campus    15116278
Name: data_level, dtype: int64


release_year
2019    6982315
2014    2596228
2018    1490175
2017    1409707
2013    1324368
2015    1313485
Name: release_year, dtype: int64


test_year
2019    3562483
2018    3419832
2014    1849170
2017    1490175
2016    1409707
2012    1324368
2015    1313485
2013     747058
Name: test_year, dtype: int64


grade_level
all    8100598
4      1428388
5      1381593
eoc    1023421
3       984805
8       910978
7       693002
6       527957
5        24651
4        24117
3        15955
6          588
8          141
7           84
Name: grade_level, dtype: int64


subject
reading           4175531
mathematics       3757577
science           2013890
writing           1739898
all_subjects      1510950
social_studies     895011
algebra_i          231189
biology   

In [13]:
columns_to_drop = ['data_release', 'data_category', 'data_level']

In [14]:
staar_df.drop(columns=columns_to_drop, inplace=True)

In [15]:
staar_df.shape

(15116278, 10)

In [16]:
show_missing(staar_df)

,num_missing,missing_percentage,num_empty,empty_percentage,nan_count,nan_percentage
release_year,0,0.0,0,0.0,0,0.0
test_year,0,0.0,0,0.0,0,0.0
campus_number,0,0.0,0,0.0,0,0.0
grade_level,0,0.0,0,0.0,0,0.0
subject,0,0.0,0,0.0,0,0.0
proficiency,0,0.0,0,0.0,0,0.0
demog,0,0.0,0,0.0,0,0.0
numerator,0,0.0,0,0.0,0,0.0
denominator,0,0.0,0,0.0,0,0.0
new_rate,0,0.0,0,0.0,0,0.0


In [17]:
show_missing(peims_df)

,num_missing,missing_percentage,num_empty,empty_percentage,nan_count,nan_percentage
DISTRICT NUMBER,0,0.0,0,0.0,0,0.0
DISTRICT NAME,0,0.0,0,0.0,0,0.0
YEAR,0,0.0,0,0.0,0,0.0
GEN FUNDS-LOCAL TAX REVENUE FROM M&O,0,0.0,0,0.0,0,0.0
ALL FUNDS-LLOCAL TAX REVENUE FROM M&O,0,0.0,0,0.0,0,0.0
GEN FUNDS-STATE REVENUE,0,0.0,0,0.0,0,0.0
ALL FUNDS-STATE REVENUE,0,0.0,0,0.0,0,0.0
GEN FUNDS-FEDERAL REVENUE,0,0.0,0,0.0,0,0.0
ALL FUNDS-FEDERAL REVENUE,0,0.0,0,0.0,0,0.0
GEN FUNDS-OTHER LOCAL & INTERMEDIATE REVENUE,0,0.0,0,0.0,0,0.0


In [18]:
staar_df.head()

,release_year,test_year,campus_number,grade_level,subject,proficiency,demog,numerator,denominator,new_rate
0,2014,2013,1902001,eoc,algebra_1,approaches,all_students,41,48,85.42
1,2014,2014,1902001,eoc,algebra_1,approaches,all_students,51,57,89.47
2,2014,2013,1902001,eoc,algebra_1,approaches,at_risk,11,17,64.71
3,2014,2014,1902001,eoc,algebra_1,approaches,at_risk,32,38,84.21
4,2014,2013,1902001,eoc,algebra_1,approaches,economic_disadvant,17,20,85.00


In [19]:
staar_df['campus_number'] = staar_df['campus_number'].apply(lambda x: '{0:0>9}'.format(x))

In [20]:
staar_df.head()

,release_year,test_year,campus_number,grade_level,subject,proficiency,demog,numerator,denominator,new_rate
0,2014,2013,001902001,eoc,algebra_1,approaches,all_students,41,48,85.42
1,2014,2014,001902001,eoc,algebra_1,approaches,all_students,51,57,89.47
2,2014,2013,001902001,eoc,algebra_1,approaches,at_risk,11,17,64.71
3,2014,2014,001902001,eoc,algebra_1,approaches,at_risk,32,38,84.21
4,2014,2013,001902001,eoc,algebra_1,approaches,economic_disadvant,17,20,85.00


In [21]:
staar_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15116278 entries, 0 to 10024060
Data columns (total 10 columns):
 #   Column         Dtype  
---  ------         -----  
 0   release_year   int64  
 1   test_year      int64  
 2   campus_number  object 
 3   grade_level    object 
 4   subject        object 
 5   proficiency    object 
 6   demog          object 
 7   numerator      int64  
 8   denominator    int64  
 9   new_rate       float64
dtypes: float64(1), int64(4), object(5)
memory usage: 1.2+ GB


In [22]:
staar_df.drop(columns=['new_rate'], inplace=True)

In [23]:
staar_df.head()

,release_year,test_year,campus_number,grade_level,subject,proficiency,demog,numerator,denominator
0,2014,2013,001902001,eoc,algebra_1,approaches,all_students,41,48
1,2014,2014,001902001,eoc,algebra_1,approaches,all_students,51,57
2,2014,2013,001902001,eoc,algebra_1,approaches,at_risk,11,17
3,2014,2014,001902001,eoc,algebra_1,approaches,at_risk,32,38
4,2014,2013,001902001,eoc,algebra_1,approaches,economic_disadvant,17,20


In [24]:
staar_df['district'] = staar_df['campus_number'].str[:6]

In [25]:
staar_df.head()

,release_year,test_year,campus_number,grade_level,subject,proficiency,demog,numerator,denominator,district
0,2014,2013,001902001,eoc,algebra_1,approaches,all_students,41,48,001902
1,2014,2014,001902001,eoc,algebra_1,approaches,all_students,51,57,001902
2,2014,2013,001902001,eoc,algebra_1,approaches,at_risk,11,17,001902
3,2014,2014,001902001,eoc,algebra_1,approaches,at_risk,32,38,001902
4,2014,2013,001902001,eoc,algebra_1,approaches,economic_disadvant,17,20,001902


In [26]:
staar_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15116278 entries, 0 to 10024060
Data columns (total 10 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   release_year   int64 
 1   test_year      int64 
 2   campus_number  object
 3   grade_level    object
 4   subject        object
 5   proficiency    object
 6   demog          object
 7   numerator      int64 
 8   denominator    int64 
 9   district       object
dtypes: int64(4), object(6)
memory usage: 1.2+ GB


In [27]:
df = staar_df.groupby(['release_year',
                  'test_year',
                  'district',
                  'grade_level',
                  'subject',
                  'proficiency',
                  'demog'
                 ])[["numerator", "denominator"]].sum().reset_index()

In [28]:
df['district'].nunique()

1258

In [29]:
df.head()

,release_year,test_year,district,grade_level,subject,proficiency,demog,numerator,denominator
0,2013,2012,001902,3,mathematics,approaches,all_students,33,42
1,2013,2012,001902,3,mathematics,approaches,at_risk,13,18
2,2013,2012,001902,3,mathematics,approaches,economic_disadvant,15,19
3,2013,2012,001902,3,mathematics,approaches,female,17,20
4,2013,2012,001902,3,mathematics,approaches,male,16,22


In [30]:
df.shape

(3676553, 9)

In [31]:
df.to_csv('../data/inter/01 - district_staar_df.csv', index=False)  

### EXPLORATION

### MODELING

# Conclusion